# Model Controller Tutorial: GPT2 model (Regression)

> This notebook contains some example of how to use the GPT2-based models in this NLP library

- skip_showdoc: true
- skip_exec: true

In this series, we walk through some of the capability of this library: single-head classification, multi-head classification, multi-label classification, and regression. If you want a more detailed tutorial, check [this](https://anhquan0412.github.io/that-nlp-library/model_classification_tutorial.html) out

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import os

In [ ]:
#This will specify a (or a list) of GPUs for training
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [ ]:
from that_nlp_library.text_transformation import *
from that_nlp_library.text_augmentation import *
from that_nlp_library.text_main import *
from that_nlp_library.utils import seed_everything

In [ ]:
from underthesea import text_normalize
from functools import partial
from pathlib import Path
import pandas as pd
import numpy as np
import nlpaug.augmenter.char as nac
from datasets import load_dataset
import random
from transformers import AutoTokenizer
from datasets import Dataset
import torch
from transformers.models.gpt2.modeling_gpt2 import GPT2Model

# Define the custom augmentation function

In [ ]:
def nlp_aug_stochastic(x,aug=None,p=0.5):
    if not isinstance(x,list): 
        if random.random()<p: return aug.augment(x)[0]
        return x
    news=[]
    originals=[]
    for _x in x:
        if random.random()<p: news.append(_x)
        else: originals.append(_x)
    # only perform augmentation when needed
    if len(news): news = aug.augment(news)
    return news+originals

In [ ]:
aug = nac.KeyboardAug(aug_char_max=3,aug_char_p=0.1,aug_word_p=0.07)
nearby_aug_func = partial(nlp_aug_stochastic,aug=aug,p=0.3)

# Create a TextDataController object

We will reuse the data and the preprocessings in [this tutorial](https://anhquan0412.github.io/that-nlp-library/text_main.html) 

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')

In [ ]:
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Rating',
                         sup_types='regression',
                         filter_dict={'Review Text': lambda x: x is not None},
                         metadatas=['Title','Division Name'],
                         content_transformations=[text_normalize,str.lower],
                         content_augmentations= [nearby_aug_func,str.lower], 
                         # add "str.lower" here because nearby_aug might return uppercase character
                         val_ratio=0.2,
                         batch_size=1000,
                         seed=42,
                         num_proc=20,
                         verbose=False
                        )

Define our tokenizer for GPT2

In [ ]:
_tokenizer = AutoTokenizer.from_pretrained('gpt2')

In [ ]:
_tokenizer.pad_token = _tokenizer.eos_token
_tokenizer.padding_side = 'left'

In [ ]:
print(_tokenizer)
print(len(_tokenizer))

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)
50257


Process and tokenize our dataset

In [ ]:
tdc.process_and_tokenize(_tokenizer,max_length=100,shuffle_trn=True)

Map (num_proc=20):   0%|          | 0/18112 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/18112 [00:00<?, ? examples/s]

Flattening the indices (num_proc=20):   0%|          | 0/18112 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/18112 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4529 [00:00<?, ? examples/s]

In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Title', 'Review Text', 'Rating', 'Division Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 18112
    })
    validation: Dataset({
        features: ['Title', 'Review Text', 'Rating', 'Division Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4529
    })
})

# Model Experiment: GPT2 Single-Head Regression

In [ ]:
from that_nlp_library.models.roberta.classifiers import ConcatHeadSimple
from that_nlp_library.models.gpt2.classifiers import *
from that_nlp_library.model_main import *
from sklearn.metrics import mean_absolute_error,mean_squared_log_error,f1_score, accuracy_score

comet_ml is installed but `COMET_API_KEY` is not set.


## Using HuggingFace model initialization

In [ ]:
from transformers.models.gpt2.modeling_gpt2 import GPT2ForSequenceClassification

In [ ]:
num_classes=1

In [ ]:
seed_everything(42)
model = GPT2ForSequenceClassification.from_pretrained('gpt2',num_labels=num_classes)
model = model.to('cuda:0')

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config.pad_token_id = model.config.eos_token_id
model.resize_token_embeddings(len(_tokenizer))

Embedding(50257, 768)

In [ ]:
metric_funcs = [mean_absolute_error,mean_squared_log_error]
controller = ModelController(model,tdc,seed=42)

In [ ]:
lr = 1e-4
bs=32
wd=0.01
epochs= 3

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics,
              )


/home/quan/anaconda3/envs/fastai_v2/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Mean Absolute Error Rating,Mean Squared Log Error Rating
1,No log,0.533235,0.505272,0.034282
2,0.876900,0.429784,0.440251,0.026974
3,0.876900,0.396184,0.430572,0.024915


## Using GPT2-based model (no concatenation), but with a custom head to limit the output range

In [ ]:
num_classes=1

In [ ]:
gpt2_body = GPT2Model.from_pretrained('gpt2')

In [ ]:
class GPT2SigmoidRange(torch.nn.Module):
    def __init__(self,
                 config,
                 high,
                 low,
                 **kwargs
                ):
        super().__init__()
        self.high=high
        self.low=low
        self.score = torch.nn.Linear(config.n_embd, config.num_labels, bias=False)
    def forward(self, inp, **kwargs):
        logits = self.score(inp)
        return torch.sigmoid(logits)*(self.high-self.low)+self.low

In [ ]:
_model_kwargs={
    # overall model hyperparams
    'head_class_sizes':num_classes,
    'head_class': GPT2SigmoidRange,
    # classfication head hyperparams
    'high':5, # the maximum rating
    'low': 1, # the minimum rating
}

model = model_init_classification(model_class = GPT2BaseForSequenceClassification,
                                  cpoint_path = 'gpt2', 
                                  output_hidden_states=False,
                                  seed=42,
                                  body_model=gpt2_body,
                                  model_kwargs = _model_kwargs)

Loading body weights. This assumes the body is the very first block of your custom architecture


In [ ]:
model.body_model.resize_token_embeddings(len(_tokenizer))

Embedding(50257, 768)

In [ ]:
metric_funcs = [mean_absolute_error,mean_squared_log_error]
controller = ModelController(model,tdc,seed=42)

In [ ]:
seed_everything(42)

lr = 1e-4
bs=32
wd=0.01
epochs= 3

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics,
              )

/home/quan/anaconda3/envs/fastai_v2/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mean Absolute Error Rating,Mean Squared Log Error Rating
1,No log,0.519211,0.533733,0.030991
2,0.652900,0.363461,0.404610,0.024032
3,0.652900,0.339726,0.399461,0.021597


## Using the GPT2 custom model (concatenation)

In [ ]:
num_classes=1

In [ ]:
gpt2_body = GPT2Model.from_pretrained('gpt2')

In [ ]:
_model_kwargs={
    # overall model hyperparams
    'head_class_sizes':num_classes,
    'head_class': ConcatHeadSimple,
    # classfication head hyperparams
    'layer2concat':4,
    'classifier_dropout':0.1 
}

In [ ]:
model = model_init_classification(model_class = GPT2HiddenStateConcatForSequenceClassification,
                                  cpoint_path = 'gpt2', 
                                  output_hidden_states=True, # since we are using 'hidden layer contatenation' technique
                                  seed=42,
                                  body_model=gpt2_body,
                                  model_kwargs = _model_kwargs)
model.body_model.resize_token_embeddings(len(_tokenizer))

Loading body weights. This assumes the body is the very first block of your custom architecture


Embedding(50257, 768)

In [ ]:
metric_funcs = [mean_absolute_error,mean_squared_log_error]
controller = ModelController(model,tdc,seed=42)

In [ ]:
seed_everything(42)

lr = 1e-4
bs=32
wd=0.01
epochs= 3

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics,
              )

/home/quan/anaconda3/envs/fastai_v2/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mean Absolute Error Rating,Mean Squared Log Error Rating
1,No log,0.867920,0.667104,0.045717
2,2.768900,0.422719,0.502831,0.026071
3,2.768900,0.381429,0.437354,0.023720


In [ ]:
controller.trainer.model.save_pretrained('./sample_weights/my_model')

## Predict Validation

In [ ]:
_model_kwargs

{'head_class_sizes': 1,
 'head_class': that_nlp_library.models.roberta.classifiers.ConcatHeadSimple,
 'layer2concat': 4,
 'classifier_dropout': 0.1}

In [ ]:
trained_model = model_init_classification(model_class = GPT2HiddenStateConcatForSequenceClassification,
                                          cpoint_path = Path('./sample_weights/my_model'), 
                                          output_hidden_states=True,
                                          seed=42,
                                          model_kwargs = _model_kwargs)

controller = ModelController(trained_model,tdc,seed=42)

In [ ]:
df_val = controller.predict_ddict(ds_type='validation')

-------------------- Start making predictions --------------------


Map:   0%|          | 0/4529 [00:00<?, ? examples/s]

Map:   0%|          | 0/4529 [00:00<?, ? examples/s]

In [ ]:
df_val = df_val.to_pandas()
df_val.head()

,Title,Review Text,Rating,Division Name,label,input_ids,attention_mask,pred_Rating
0,,general . . this picture doesn't do the skirt ...,5.0,general,5.0,"[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.318881
1,,"general . . easy to wear ! cute , comfy ... wi...",4.0,general,4.0,"[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.838667
2,,"general . . nice sweater , just did not look g...",3.0,general,3.0,"[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2.627621
3,nice cropped jacket,general . nice cropped jacket . this jacket wa...,5.0,general,5.0,"[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.588667
4,great dress!,general petite . great dress ! . i wasn't plan...,5.0,general petite,5.0,"[24622, 4273, 578, 764, 1049, 6576, 5145, 764,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",4.865950


You can try to get your metric to see if it matches your last traing epoch's above

In [ ]:
mean_absolute_error(df_val['label'],df_val['pred_Rating'])

0.43732401377898383

In [ ]:
mean_squared_log_error(df_val['label'],df_val['pred_Rating'])

0.02372025958062093

## Predict Test set

We will go through details on how to make a prediction on a completely new and raw dataset using our trained model. For now, let's reuse the sample csv and pretend it's our test set

In [ ]:
df_test = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig').sample(frac=0.2,random_state=1)


# save the label, as we will calculate some metrics later
true_labels = df_test[~df_test['Review Text'].isna()].Rating.values 

# drop the label (you don't need to, but this is necessary to simulate an actual test set)
df_test.drop('Rating',axis=1,inplace=True)

In [ ]:
_test_dset = Dataset.from_pandas(df_test)
_test_dset_predicted = controller.predict_raw_dset(_test_dset,
                                                   do_filtering=True, # since we have some text filtering in the processing
                                                  )

Filter (num_proc=20):   0%|          | 0/4697 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4533 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4533 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4533 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4533 [00:00<?, ? examples/s]

-------------------- Start making predictions --------------------


Map:   0%|          | 0/4533 [00:00<?, ? examples/s]

Map:   0%|          | 0/4533 [00:00<?, ? examples/s]

In [ ]:
df_test_predicted = _test_dset_predicted.to_pandas()

In [ ]:
df_test_predicted.head()

,Title,Review Text,Division Name,input_ids,attention_mask,pred_Rating
0,perfect for work and play,general . perfect for work and play . this shi...,general,"[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.907113
1,,general petite . . i don't know why i had the ...,general petite,"[24622, 4273, 578, 764, 764, 1312, 836, 470, 7...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",4.097093
2,great pants,general petite . great pants . thes e cords ar...,general petite,"[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5.009317
3,surprisingly comfy for a button down,general petite . surprisingly comfy for a butt...,general petite,"[24622, 4273, 578, 764, 12362, 401, 24928, 329...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",4.802028
4,short and small,general petite . short and small . the shirt i...,general petite,"[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2.142920


Let's quickly check the score to make sure everything works correctly

In [ ]:
mean_absolute_error(true_labels,df_test_predicted['pred_Rating'])

0.4236907030529937

In [ ]:
mean_squared_log_error(true_labels,df_test_predicted['pred_Rating'])

0.02372909015123248

In [ ]:
raw_content={'Review Text': 'This shirt is so comfortable I love it!',
             'Title': 'Great shirt',
             'Division Name': 'general'}
raw_content

{'Review Text': 'This shirt is so comfortable I love it!',
 'Title': 'Great shirt',
 'Division Name': 'general'}

In [ ]:
df_result = controller.predict_raw_text(raw_content)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

-------------------- Start making predictions --------------------


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
df_result

,Review Text,Title,Division Name,input_ids,attention_mask,pred_Rating
0,general . great shirt . this shirt is so comfo...,great shirt,general,"[24622, 764, 1049, 10147, 764, 428, 10147, 318...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",4.838715


# Model Experiment: GPT2 Multi-Head Regression

In [ ]:
from that_nlp_library.models.roberta.classifiers import ConcatHeadSimple
from that_nlp_library.models.gpt2.classifiers import *
from that_nlp_library.model_main import *
from sklearn.metrics import mean_absolute_error,mean_squared_log_error,f1_score, accuracy_score

comet_ml is installed but `COMET_API_KEY` is not set.


## Re-define the TextDataController

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')


In [ ]:
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names=['Rating','Department Name'],
                         sup_types=['regression','classification'],
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None},
                         metadatas=['Title'],
                         content_transformations=[text_normalize,str.lower],
                         content_augmentations= [nearby_aug_func,str.lower], 
                         val_ratio=0.2,
                         batch_size=1000,
                         seed=42,
                         num_proc=20,
                         verbose=False
                        )

In [ ]:
_tokenizer = AutoTokenizer.from_pretrained('gpt2')

In [ ]:
_tokenizer.pad_token = _tokenizer.eos_token
_tokenizer.padding_side = 'left'

Process and tokenize our dataset

In [ ]:
tdc.process_and_tokenize(_tokenizer,max_length=100,shuffle_trn=True)

Filter (num_proc=20):   0%|          | 0/18102 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/18101 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/18101 [00:00<?, ? examples/s]

Flattening the indices (num_proc=20):   0%|          | 0/18101 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/18101 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4526 [00:00<?, ? examples/s]

In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['Title', 'Review Text', 'Rating', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 18101
    })
    validation: Dataset({
        features: ['Title', 'Review Text', 'Rating', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

In [ ]:
tdc.label_lists

[[], ['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend']]

## Using the GPT2 custom model (concatenation)

In [ ]:
num_classes=[1,len(tdc.label_lists[1])] # 1 head size 1 for regression, 1 head size 6 for classification
num_classes

[1, 6]

In [ ]:
gpt2_body = GPT2Model.from_pretrained('gpt2')

In [ ]:
_model_kwargs={
    # overall model hyperparams
    'head_class_sizes':num_classes,
    'head_class': ConcatHeadSimple,
    # classfication head hyperparams
    'layer2concat':3,
    'classifier_dropout':0.1 
}

In [ ]:
model = model_init_classification(model_class = GPT2HiddenStateConcatForSequenceClassification,
                                  cpoint_path = 'gpt2', 
                                  output_hidden_states=True,
                                  seed=42,
                                  body_model=gpt2_body,
                                  model_kwargs = _model_kwargs)

Loading body weights. This assumes the body is the very first block of your custom architecture


In [ ]:
model.body_model.resize_token_embeddings(len(_tokenizer))

Embedding(50257, 768)

If you use `multihead` and each head does a different supervised learning type (as in this case when you have both classification and regression head), and you have separate metric for each type, you must define a `metric_types` to let the controller knows what metric functions to apply for each head

In [ ]:
metric_funcs = [mean_absolute_error,partial(f1_score,average='macro'),accuracy_score]
metric_types = ['regression','classification','classification']

In [ ]:
controller = ModelController(model,tdc,seed=42)

In [ ]:
seed_everything(42)

lr = 1e-4
bs=32
wd=0.01
epochs= 3

controller.fit(epochs,lr,
               metric_funcs=metric_funcs,
               metric_types=metric_types,
               batch_size=bs,
               weight_decay=wd,
               save_checkpoint=False,
               compute_metrics=compute_metrics,
              )

Epoch,Training Loss,Validation Loss,Mean Absolute Error Rating,F1 Score Department name,Accuracy Score Department name
1,No log,1.699756,0.846051,0.528974,0.808882
2,3.776500,0.942072,0.510461,0.620426,0.855722
3,3.776500,0.864971,0.454651,0.632368,0.858816


In [ ]:
controller.trainer.model.save_pretrained('./sample_weights/my_model')

## Predict Validation

In [ ]:
_model_kwargs

{'head_class_sizes': [1, 6],
 'head_class': that_nlp_library.models.roberta.classifiers.ConcatHeadSimple,
 'layer2concat': 3,
 'classifier_dropout': 0.1}

In [ ]:
trained_model = model_init_classification(model_class = GPT2HiddenStateConcatForSequenceClassification,
                                          cpoint_path = Path('./sample_weights/my_model'), 
                                          output_hidden_states=True,
                                          seed=42,
                                          model_kwargs = _model_kwargs)

controller = ModelController(trained_model,tdc,seed=42)

In [ ]:
df_val = controller.predict_ddict(ds_type='validation')

-------------------- Start making predictions --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

In [ ]:
df_val = df_val.to_pandas()
df_val.head()

,Title,Review Text,Rating,Department Name,label,input_ids,attention_mask,pred_Rating,pred_Department Name,pred_prob_Department Name
0,,. such a fun jacket ! great to wear in the spr...,5.0,Intimate,"[5.0, 2.0]","[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.865183,Jackets,0.909341
1,simple and elegant,simple and elegant . i thought this shirt was ...,5.0,Tops,"[5.0, 4.0]","[36439, 290, 19992, 764, 1312, 1807, 428, 1014...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",4.395523,Tops,0.993128
2,retro and pretty,retro and pretty . this top has a bit of a ret...,5.0,Tops,"[5.0, 4.0]","[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.575636,Tops,0.983207
3,summer/fall wear,summer / fall wear . i first spotted this on a...,5.0,Dresses,"[5.0, 1.0]","[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...",4.901958,Dresses,0.912590
4,perfect except slip,perfect except slip . this is my new favorite ...,4.0,Dresses,"[4.0, 1.0]","[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.628626,Dresses,0.988459


You can try to get your metric to see if it matches your last traing epoch's above

In [ ]:
mean_absolute_error(df_val['Rating'],df_val['pred_Rating'])

0.4546500524745491

In [ ]:
f1_score(df_val['Department Name'],df_val['pred_Department Name'],average='macro')

0.632367541709813

## Predict Test set

We will go through details on how to make a prediction on a completely new and raw dataset using our trained model. For now, let's reuse the sample csv and pretend it's our test set

In [ ]:
df_test = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig').sample(frac=0.2,random_state=1)

# drop the label (you don't need to, but this is necessary to simulate an actual test set)
df_test.drop(['Rating','Department Name'],axis=1,inplace=True)

In [ ]:
_test_dset = Dataset.from_pandas(df_test)
_test_dset_predicted = controller.predict_raw_dset(_test_dset,
                                                   do_filtering=True, # since we have some text filtering in the processing
                                                   topk=3
                                                  )

Filter (num_proc=20):   0%|          | 0/4697 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4533 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4533 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4533 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/4533 [00:00<?, ? examples/s]

-------------------- Start making predictions --------------------


Map:   0%|          | 0/4533 [00:00<?, ? examples/s]

Map:   0%|          | 0/4533 [00:00<?, ? examples/s]

Map:   0%|          | 0/4533 [00:00<?, ? examples/s]

In [ ]:
df_test_predicted = _test_dset_predicted.to_pandas()

In [ ]:
df_test_predicted.head()

,Title,Review Text,input_ids,attention_mask,pred_Rating,pred_Department Name,pred_prob_Department Name
0,perfect for work and play,perfect for work and play . this shirt works f...,"[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.979549,"[Tops, Intimate, Jackets]","[0.9960622, 0.0023680637, 0.0013514926]"
1,,. i don't know why i had the opposite problem ...,"[13, 1312, 836, 470, 760, 1521, 1312, 550, 262...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",3.823210,"[Bottoms, Intimate, Dresses]","[0.91915536, 0.048780475, 0.014232734]"
2,great pants,great pants . thes e cords are great--lightwei...,"[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.931431,"[Bottoms, Intimate, Jackets]","[0.9697613, 0.028874332, 0.0010721061]"
3,surprisingly comfy for a button down,surprisingly comfy for a button down . i am a ...,"[41199, 401, 24928, 329, 257, 4936, 866, 764, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",4.499933,"[Tops, Intimate, Jackets]","[0.9328854, 0.031557593, 0.030198798]"
4,short and small,short and small . the shirt is mostly a thick ...,"[50256, 50256, 50256, 50256, 50256, 50256, 502...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2.394195,"[Tops, Intimate, Jackets]","[0.9751625, 0.018599715, 0.0047323317]"


In [ ]:
raw_content={'Review Text': 'This shirt is so comfortable I love it!',
             'Title': 'Great shirt'}
raw_content

{'Review Text': 'This shirt is so comfortable I love it!',
 'Title': 'Great shirt'}

In [ ]:
df_result = controller.predict_raw_text(raw_content)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

-------------------- Start making predictions --------------------


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
df_result

,Review Text,Title,input_ids,attention_mask,pred_Rating,pred_Department Name,pred_prob_Department Name
0,great shirt . this shirt is so comfortable i l...,great shirt,"[18223, 10147, 764, 428, 10147, 318, 523, 6792...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",4.773216,Tops,0.982727
